In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

In [ ]:
cd KoBERT

In [ ]:
!pip install -r requirements.txt
!pip install .
!pip install torch

# transformers 의 경우 최신 버전으로 설치 시 에러 발생
!pip install transformers #==3

In [ ]:
import os
import pandas as pd
import numpy as np
import csv 
import sys

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader

# from kobert_transformers import get_kobert_model, get_distilkobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook

# 환경 세팅

In [ ]:
# pytorch 에러 확인을 위한 패키지 버전 확인
print(sys.version)
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

In [ ]:
#GPU 사용을 위해
device = torch.device("cuda:0")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# tokenizer

In [ ]:
# 이미 pre train 된 bert 모델과 vocab을 불러옴
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 위에서 불러온 vocab을 이용하여 토크나이저 선언
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# bert 모델 최적화

In [ ]:
# Setting parameters
max_len = 128 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
# 학습할 때 사용할 데이터셋 클래스 선언
# list 대신 dataFrame을 이용했기 때문에 이에 맞게 매개 변수 등을 수정함

# 기존 코드
# class BERTDataset(Dataset):
#     def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
#                  pad, pair):
#         transform = nlp.data.BERTSentenceTransform(
#             bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

#         self.sentences = [transform([i[sent_idx]]) for i in dataset]
#         self.labels = [np.int32(i[label_idx]) for i in dataset]

#     def __getitem__(self, i):
#         return (self.sentences[i] + (self.labels[i], ))

#     def __len__(self):
#         return (len(self.labels))

class BERTDataset(Dataset):
    # sentences 의 index와 label index 매개변수 제거
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

      # sentences 리스트와 label 리스트를 만드는 코드 변경
        self.sentences = [transform([i]) for i in dataset['contents']]
        self.labels = list(dataset['label'])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, # Dimensionality of the encoder layers and the pooler layer
                 num_classes = 2, # 분류할 class label의 개수                
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# 데이터 로딩

In [ ]:
train_data_path = "/content/drive/MyDrive/forColab/train1_data.csv"
train_data = pd.read_csv(train_data_path)
print('train data shape:', train_data.shape)
print('train data의 \'contents\' data type:', train_data['contents'].dtypes)
print('train data의 \'label\' data type:', train_data['label'].dtypes)
print('train data label:', train_data['label'].unique())

In [ ]:
test_data_path = "/content/drive/MyDrive/forColab/test_data.csv"
test_data = pd.read_csv(test_data_path)
print('test data shape:', test_data.shape)
print('test data의 \'contents\' data type:', test_data['contents'].dtypes)
print('test data의 \'label\' data type:', test_data['label'].dtypes)
print('test data label:', test_data['label'].unique())

# 오류있는 경우가 있어 추가
test_data = test_data[test_data['label'] != '#NAME?']
test_data['label'] = test_data['label'].astype('int')
print('test data의 \'label\' data type:', test_data['label'].dtypes)
print('test data label:', test_data['label'].unique())

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_dataset = BERTDataset(train_data, tok, max_len, True, False)
test_dataset = BERTDataset(test_data, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=5, shuffle=True)

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
model_save_path = '/content/drive/MyDrive/forColab/learned_model/'

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        # optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()         # 그래디언트 초기화
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    
    torch.save({
     'epoch' : e+1,
     'model_state_dict': model.state_dict(),
     'optimizer_state_dict': optimizer.state_dict()
     }, model_save_path + 'model_{}.tar'.format(e+1) )
    

# 모델 저장~
torch.save(model, model_save_path + 'model3.pt')  # 전체 모델 저장
torch.save(model.state_dict(), model_save_path + 'model3_state_dict.pt')  # 모델 객체의 state_dict 저장

In [ ]:
label